In [1]:
#This class store the internsity and pitch 
class Pause_Info:
  def __init__(self,intensity,stinten,pitch,stpit,max_inten,min_inten,max_pitch,min_pitch,performance,id,jokeName):
    self.intensity = intensity
    self.stinten= stinten
    self.pitch = pitch
    self.stpit= stpit
    self.max_inten=max_inten
    self.min_inten=min_inten
    self.max_pitch=max_pitch
    self.min_pitch=min_pitch
    self.performance=performance
    self.id=id
    self.jokeName=jokeName
    self.midjoke=-100
    
    
#this function loop through the folder "joke_name_matching" and make a dictionary of 
#joke id and joke name for each performance
def getJokeNameForEachPerformance(folderPath):
    os.chdir(folderPath)
    
    jokeNameDict={}
    
    for file in glob.glob("*.txt"):
        idDict={}
        filepath=folderPath+'\\'+file
        f = open(filepath, "r")
        key=f.readline()
        key=key.replace("\x00","")
        key=key.replace("\n","")
#         print(key)
        content=f.read();
        f.close()
        content=content.replace("\x00","")
        content=content.replace("\n\n","\n")
        if not content:
            continue
        lines=content.split('\n')
        lines.pop(0)
        lines.pop(-1)
        for line in lines:
            jokename=line.split(':')[0]
            id=int(line.split(':')[1].split('.')[0].split('_')[1])
#             print(jokename,id)
            idDict[id]=jokename
        jokeNameDict[key]=idDict
    
    return jokeNameDict
            

#This function extract intensity and pitch info from txt file and put all info in to a dictionary
def readTxT(path):
    f = open(path, "r")
    line = f.readline()
    
    #store the joke title
    Title = line.split('\\')[-2]
    Title=Title.replace("\x00","")
    joke_dict = {}

    content=f.read();
    f.close()
    
    #replace all these werid \x00 in python
    #also replace all the extra newline char
    content=content.replace("\x00","")
    content=content.replace("\n\n","\n")
    lines=content.split('\n')
    
    #remove the first and last extra new line char
    lines.pop(0)
    lines.pop(-1)

    count=0
    intensity=0
    stinten=0
    pitch=0
    stpit=0
    max_inten=0
    min_inten=0
    max_pitch=0
    min_pitch=0
    digits=2
    
    #store 7 info of each audio
    for line in lines:
        if(count%9==0):
            name=line.split('.')[0]
            name=name.split('_')[-1]
            id=int(name)
        elif(count%9==1):
            intensity=round(float(line),digits)
        elif(count%9==2):
            stinten=round(float(line),digits)
        elif(count%9==3):
            min_inten=round(float(line),digits)
        elif(count%9==4):
            max_inten=round(float(line),digits)
        elif(count%9==5):
            pitch=round(float(line),digits)
        elif(count%9==6):
            stpit=round(float(line),digits)   
        elif(count%9==7):
            max_pitch=round(float(line),digits)
        else:
            min_pitch=round(float(line),digits)
            if(Title not in jokeNameDict):
                break
            idDict=jokeNameDict[Title]
            jokeName=idDict[id]
            #print(jokeName)
            joke_dict[jokeName]=Pause_Info(intensity,stinten,pitch,stpit,max_inten,min_inten,max_pitch,min_pitch,Title,id,jokeName)
            if(Title not in perNameDict):
                perNameDict[Title]=1;
                
            key=Title+"-"+str(id)
            joke_dict[jokeName].midjoke=humanRateDict[key]
        # print(Title,id)
        count+=1
    return joke_dict


import glob, os



#in combine_joke_dict,
#key of combine_joke_dict represent the joke id
#value of combine_joke_dict is an array of 'Pause_Info' who are the same joke but in different perofrmance
#len of value represent how many different perofrmance contain this joke 
def findAlltxtAndCombine(folderPath):
    os.chdir(folderPath)
    combine_joke_dict={}
    
    #find all txt file inside folder
    for file in glob.glob("*.txt"):
        filepath=folderPath+'\\'+file
        single_joke_dict=readTxT(filepath)   
        for x in single_joke_dict:
            y=single_joke_dict[x]
            if x in combine_joke_dict:
                combine_joke_dict[x].append(y)
            else:
                combine_joke_dict[x]=[y]
    return combine_joke_dict         

In [2]:
%matplotlib inline

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import scale
from numpy import random, float
# plt.rcParams['font.sans-serif']=['SimHei']
# plt.rcParams['axes.unicode_minus']=False

#info_arr represent an array of same joke in different performances
#choice=1 is using intensity and pitch as x-y axis
#choice=2 is using std intensity and std pitch as x-y axis
def clusterDataIn2D(Info_arr,choice):
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    performanceName=[]
    Xname,Yname='',''
    if(choice==1):
        for joke in Info_arr:
            X=joke.intensity
            Y=joke.pitch
            data.append((X,Y))
            performanceName.append(joke.performance)
            x.append(X)
            y.append(Y)
            Xname="intensity"
            Yname="pitch"
    elif(choice==2):
        for joke in Info_arr:
            X=joke.stinten
            Y=joke.stpit
            data.append((X,Y))
            performanceName.append(joke.performance)
            x.append(X)
            y.append(Y)
            Xname="standard deviation intensity"
            Yname="standard deviation pitch"
    
    #if the data set is smller than 2,we will ignore it.
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1

    dataTitle="Data values: ("+Xname+","+Yname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    #print(jokeid)
    txtName="../jokeoutput/2D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
#     print(jokeid)
#     print (model.cluster_centers_)
    
    plt.figure(figsize=(10,6))
    plt.scatter(x, y, c=model.labels_.astype(float))
    plt.title("Joke Name: "+str(jokeid))
    plt.xlabel(Xname)
    plt.ylabel(Yname)
    
    pngName="../jokeoutput/2D/"+str(jokeid)+".png"
    
    #remove the existing png file
    if os.path.isfile(pngName):
        os.remove(pngName) 
    
    #save as png file
    plt.savefig(pngName)
    plt.show()
    

#this function will split data into 2 groups by using 4 features
def clusterDataIn4D(Info_arr):
    #replace all the '/' to '-' since it will mess up with path of windows
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    z=[]
    h=[]
    performanceName=[]
    Xname,Yname='',''
    
    #get all intensity,pitch,std intensity,std pitch
    for joke in Info_arr:
        X=joke.intensity
        Y=joke.pitch
        Z=joke.stinten
        H=joke.stpit
        data.append((X,Y,Z,H))
        performanceName.append(joke.performance)
        x.append(X)
        y.append(Y)
        z.append(Z)
        h.append(H)
        Xname="intensity"
        Yname="pitch"
        Zname="std intensity"
        Hname="std pitch"

    
    #if data set is smaller than 2, we cannot split them into 2groups
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1
    
    dataTitle="Data values: ("+Xname+","+Yname+","+Zname+","+Hname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    txtName="../jokeoutput/4D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
#     print(jokeid)
#     print (model.cluster_centers_)


#this function will split data into 2 groups by using 6 features
def clusterDataIn8D(Info_arr):
    #replace all the '/' to '-' since it will mess up with path of windows
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    z=[]
    d=[]
    e=[]
    f=[]
    g=[]
    h=[]
    performanceName=[]
    Xname,Yname='',''
    
    #get all intensity,pitch,std intensity,std pitch
    for joke in Info_arr:
        X=joke.intensity
        Y=joke.pitch
        Z=joke.stinten
        D=joke.stpit
        E=joke.max_inten
        F=joke.min_inten
        G=joke.max_pitch
        H=joke.min_pitch
 
        data.append((X,Y,Z,D,E,F,G,H))
        performanceName.append(joke.performance+"----Joke ID: "+str(joke.id))
        x.append(X)
        y.append(Y)
        z.append(Z)
        d.append(d)
        e.append(E)
        f.append(F)
        g.append(G)
        h.append(H)
        
        
#         print(E,F)
        
        Xname="intensity"
        Yname="pitch"
        Zname="std intensity"
        Dname="std pitch"
        Ename="max inten"
        Fname="min inten"
        Gname="max pitch"
        Hname="min pitch"

    
    #if data set is smaller than 2, we cannot split them into 2groups
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1
    
    dataTitle="Data values: ("+Xname+","+Yname+","+Zname+","+Dname+","+Ename+","+Fname+","+Gname+","+Hname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name and Joke ID: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name and Joke ID: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    txtName="../jokeoutput/8D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
#     print(jokeid)
#     print (model.cluster_centers_)

In [3]:
import numpy
from scipy.stats import pearsonr
import matplotlib.patches as mpatches

def overallCorrelation(combine_joke_dict):
    x=[]
    y=[]
    z=[]
    d=[]
    e=[]
    f=[]
    g=[]
    h=[]
    midjokeRating=[]
    for i in combine_joke_dict:
        arr=combine_joke_dict[i]
        for joke in arr:
            x.append(joke.intensity)
            y.append(joke.pitch)
            z.append(joke.stinten)
            d.append(joke.stpit)
            e.append(joke.max_inten)
            f.append(joke.min_inten)
            g.append(joke.max_pitch)
            h.append(joke.min_pitch)
            midjokeRating.append(joke.midjoke)
    
    arr=[x,y,z,d,e,f,g,h]
    result={}
    var=['intensity','pitch','stdIntensity','stdPitch','maxIntensity','minIntensity','maxPitch','minPitch']
    length=len(var)
    
    pontential="The correlation coefficient are higher than 7:\n"
    
    for i in range(0,length-1):
        for j in range(i+1,length):
            #save txt result
            key=var[i]+'-'+var[j]
            result[key]=pearsonr(arr[i],arr[j])[0]
            if(abs(result[key])>0.7):
                pontential=pontential+key+": "+str(result[key])+"\n"
            
            #draw plot
            path="../jokeoutput/overall-correlation/"+key+".png"            
            draw2Dplot(var[i],var[j],arr[i],arr[j],path,midjokeRating)

    print(len(result))
    txtName="../jokeoutput/overall-correlation/correlation.txt"
    
    output=''
    for n in result:
        output=output+n+": "+str(result[n])+"\n"

    output=output+"\n\n"+pontential
    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(output)
    f.close()
    
def jokeWiseCorrelation(combine_joke_dict):
    x=[]
    y=[]
    z=[]
    d=[]
    e=[]
    f=[]
    g=[]
    h=[]
    midjokeRating=[]
    var=['intensity','pitch','stdIntensity','stdPitch','maxIntensity','minIntensity','maxPitch','minPitch']
    length=len(var)
    pontential="The correlation coefficient are higher than 7:\n"
    for i in combine_joke_dict:
        arr=combine_joke_dict[i]
        x.clear()
        y.clear()
        z.clear()
        d.clear()
        e.clear()
        f.clear()
        g.clear()
        h.clear()
        midjokeRating.clear()
        jokename=''
        for joke in arr:
            x.append(joke.intensity)
            y.append(joke.pitch)
            z.append(joke.stinten)
            d.append(joke.stpit)
            e.append(joke.max_inten)
            f.append(joke.min_inten)
            g.append(joke.max_pitch)
            h.append(joke.min_pitch) 
            jokename=joke.jokeName
            midjokeRating.append(joke.midjoke)
            
        jokename=jokename.replace('/','-')
        arr=[x,y,z,d,e,f,g,h]
        result={}
        dirPath="../jokeoutput/jokewise-correlation/"+jokename
                
        for i in range(0,length-1):
            for j in range(i+1,length):
                #save txt result
                key=var[i]+'-'+var[j]
#                 print(len(arr[i]),len(arr[j]))
                if(len(arr[i])<10):
                    continue
                    
                    
                else:    
                    if not os.path.exists(dirPath):
                        os.mkdir(dirPath)
                    result[key]=pearsonr(arr[i],arr[j])[0]
                    if(abs(result[key])>0.7):
                        pontential=pontential+key+": "+str(result[key])+"\n"
                    #draw plot
                    path=dirPath+"/"+key+".png"           
#                     print(path)
                    draw2Dplot(var[i],var[j],arr[i],arr[j],path,midjokeRating)

        txtName=dirPath+"/correlation.txt"

        output=''
        for n in result:
            output=output+n+": "+str(result[n])+"\n"
        if(output==''):
            continue
        output=output+"\n\n"+pontential
        if os.path.isfile(txtName):
            os.remove(txtName) 
        file = open(txtName, "a")
        file.write(output)
        file.close()
        pontential="The correlation coefficient are higher than 7:\n"
    
def draw2Dplot(Xname,Yname,x,y,path,midjokeRating):
    fig=plt.figure(figsize=(10,6))
#     plt.scatter(x, y)
    plt.title(Xname+"-"+Yname)
    plt.xlabel(Xname)
    plt.ylabel(Yname)
    for i in range(0,len(x)):
        if(midjokeRating[i]==1):
            plt.scatter(x[i],y[i],marker="^",color='r',label='Blue stars')
        elif(midjokeRating[i]==0):
            plt.scatter(x[i],y[i],marker="x",color='b',label='Red stars')
            
    red_patch = mpatches.Patch(color='red', label='mid joke happened')
    blue_patch = mpatches.Patch(color='blue', label='mid joke did not happen')
    plt.legend(handles=[red_patch,blue_patch])
    
    pngName=path
    #remove the existing png file
    if os.path.isfile(pngName):
        os.remove(pngName) 
    #save as png file
    plt.savefig(pngName)
#     plt.show()
    plt.close(fig) 

def humanAnnotation(path):
    humanRateDict={}
    f = open(path, "r")
    lines = f.readlines()
    value=0
    key=''
    for line in lines:
        if(".eaf" in line):
            if(key is not ''):
                humanRateDict[key]=value
                key=''
                value=0
                
            arr=line.split('\\')
            if("\\Annotations\\" not in line):
                key=arr[-2]+'-'+arr[-1].split('.')[-2].split('_')[-1]
            else:
                key=arr[-3]+'-'+arr[-1].split('.')[-2].split('_')[-1]
        elif(not line.isspace()):
            value=1
            
    if(key not in humanRateDict):
        humanRateDict[key]=value
#     for x in humanRateDict:
#         print(x,humanRateDict[x])
    return humanRateDict

def drawBoxPlot(a,b,title,path):
    data=[a,b]
    fig1, ax1 = plt.subplots(figsize=(10,10))
    ax1.set_title(title)
    red_square = dict(markerfacecolor='r', marker='s')
    ax1.boxplot(data,flierprops=red_square)
    plt.xticks([1, 2], ['Laughter not happen(0)', 'Laughter happen(1)'])
    pngName=path+title+".png"
    plt.savefig(pngName)
    plt.close()

def overallBoxPlot(combine_joke_dict):
    group0=[]
    group1=[]
    for i in combine_joke_dict:
            arr=combine_joke_dict[i]
            for joke in arr:
                val=joke.midjoke
                if(val==0):
                    group0.append(joke)
                else:
                    group1.append(joke)
    name=['intensity,dB','pitch,Hz','std Intensity,dB','std Pitch,Hz','max_intensity,dB','min_intensity,dB','max_pitch,Hz','min_pitch,Hz']
    var=['intensity','pitch','stinten','stpit','max_inten','min_inten','max_pitch','min_pitch']
    path="../jokeoutput/overall-boxplot/"
    
    index=0
    for feature in var:
        a=[]
        b=[]
        for joke in group0:
            a.append(getattr(joke,feature))
        for joke in group1:
            b.append(getattr(joke,feature))
        drawBoxPlot(a,b,name[index],path) 
        index=index+1

def jokeWiseBoxPlot(combine_joke_dict):
    for i in combine_joke_dict:
        arr=combine_joke_dict[i]
        if(len(arr)<10):
            continue
        group0=[]
        group1=[]
        jokename=''
        for joke in arr:
            val=joke.midjoke
            jokename=joke.jokeName.replace('/','-')
            if(val==0):
                group0.append(joke)
            else:
                group1.append(joke)
        name=['intensity,dB','pitch,Hz','std Intensity,dB','std Pitch,Hz','max_intensity,dB','min_intensity,dB','max_pitch,Hz','min_pitch,Hz']
        var=['intensity','pitch','stinten','stpit','max_inten','min_inten','max_pitch','min_pitch']
        path="../jokeoutput/jokewise-boxplot/"+jokename
        if not os.path.exists(path):
            os.mkdir(path)
            
        index=0
        for feature in var:
            a=[]
            b=[]
            for joke in group0:
                a.append(getattr(joke,feature))
            for joke in group1:
                b.append(getattr(joke,feature))
            drawBoxPlot(a,b,name[index],path+'/') 
            index=index+1
    



In [4]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from sklearn import tree
from os import system
import pydot
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import numpy as np
from mlxtend.plotting import plot_decision_regions
from combo.models.classifier_comb import SimpleClassifierAggregator
from combo.utils.data import evaluate_print
from sklearn.ensemble import RandomForestClassifier

def leaveOnePerformanceOut(combine_joke_dict,perName):
    train_data={}
    valid_data={}
    for key in combine_joke_dict:
        jokearr=combine_joke_dict[key]
        for joke in jokearr:
            if(joke.performance==perName):
                if(key in valid_data):
                    valid_data[key].append(joke)
                else:
                    valid_data[key]=[joke]
            else:
                if(key in train_data):
                    train_data[key].append(joke)
                else:
                    train_data[key]=[joke]
    return train_data,valid_data

def decision_tree(train_data,valid_data,overall):
    features=['intensity','pitch','stdIntensity','stdPitch','maxIntensity','minIntensity','maxPitch','minPitch']
    X=[]
    Y=[]

    for i in train_data:
        arr=train_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            X.append(singleSample)
            Y.append(joke.midjoke)
    clf = tree.DecisionTreeClassifier()
    clf.fit(X, Y)
    
#     tree.plot_tree(clf.fit(X, Y)) 
#     dotfile = open("../jokeoutput/decision-tree/dtree2.dot", 'w')
    
#     tree.export_graphviz(clf, out_file = dotfile, feature_names = features)
#     dotfile.close()
#     (graph,) = pydot.graph_from_dot_file('../jokeoutput/decision-tree/dtree2.dot')
#     graph.write_png('../jokeoutput/decision-tree/result.png')
    
    
    validX=[]
    validY=[]
    
    perName=""
    for i in valid_data:
        arr=valid_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            validX.append(singleSample)
            validY.append(joke.midjoke)
            perName=joke.performance
    
    clf2 = RandomForestClassifier(n_estimators=100)
    clf2 = clf2.fit(X, Y)
    if(overall==1):   
        result1="onetree: "+str(clf.score(validX, validY))
        
        result2="random forest: "+str(clf2.score(validX, validY))
        txtFile = open("../jokeoutput/decision-tree/tree_result.txt", 'a')
        txtFile.write(perName+'\n'+result1+'\n\n')
        txtFile.close()
        txtFile = open("../jokeoutput/decision-tree/forest_result.txt", 'a')
        txtFile.write(perName+'\n'+result2+'\n\n')
        txtFile.close()
    #     print(result1+'\n'+result2)

    #     evaluate_print('Decision Tree        |', validY, clf.predict(validX))
    #     evaluate_print('Random Forest       |', validY, clf2.predict(validX))
    
    return clf,clf2,clf.score(validX, validY),clf2.score(validX, validY)

    
#for KNN, i have train_data1,train_data2,valid_data. Since I dont know what is the best number for K.
#i use train_data1 to train model, then I have one set of performance which is train_data2 to get the Accuracy.
#in the end, i apply the best K i found into valid_data to get the final Accuracy
def KNN(train_data1,train_data2,valid_data,overall):

    X=[]
    Y=[]
    perName=""
    for i in train_data1:
        arr=train_data1[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            X.append(singleSample)
            Y.append(joke.midjoke)
    
    X2=[]
    Y2=[]
    for i in train_data2:
        arr=train_data2[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            X2.append(singleSample)
            Y2.append(joke.midjoke)

    maxscore=0
    bestclf=KNeighborsClassifier(n_neighbors=3)
    bestclf.fit(X, Y)
    for n in range(3,int(len(train_data1)/2),2):
        clf = KNeighborsClassifier(n_neighbors=n)
        clf.fit(X, Y)
        score=clf.score(X2, Y2)
        if(score>maxscore):
            bestclf=clf
            bestN=n
            maxscore=score
    
    clf=bestclf
#     print(maxscore,bestN)
    validX=[]
    validY=[]
    
    for i in valid_data:
        arr=valid_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            validX.append(singleSample)
            validY.append(joke.midjoke)
            perName=joke.performance
#     print(validX)
#     print(validY)
    
    if(overall==1):  
        result=perName+"\nBest Neighbor Number: "+str(bestN)+'\n'
        result=result+"KNN Accuracy: "+str(bestclf.score(validX, validY))+"\n\n"
    #     print(result)

    #     evaluate_print('KNN        |', validY, clf.predict(validX))

        txtFile = open("../jokeoutput/KNN/result.txt", 'a')
        txtFile.write(result)
        txtFile.close()
        
    return clf,bestclf.score(validX, validY)

def SVC_Algorithm(train_data,valid_data,overall):
    X=[]
    Y=[]
    
    for i in train_data:
        arr=train_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            X.append(singleSample)
            Y.append(joke.midjoke)
            
    if(not(1 in Y and 0 in Y)):
        return None,None,None,None,None,None
        
    rbfclf = SVC(kernel='rbf',C=10,gamma=2**-5)
    rbfclf.fit(X, Y)
    
    linearclf = SVC(kernel='linear')
    linearclf.fit(X, Y)
    
    polyclf = SVC(kernel='poly',degree=8)
    polyclf.fit(X, Y)
    
    validX=[]
    validY=[]
    perName=''
    for i in valid_data:
        arr=valid_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            validX.append(singleSample)
            validY.append(joke.midjoke)
            perName=joke.performance
    
    score1,score2,score3=0,0,0
    
    if(overall==1):
        score1=rbfclf.score(validX, validY)
        result1="RBF accuracy:" +str(score1)
    #     print(result1)
        score2=linearclf.score(validX, validY)
        result2="Linear accuracy:" +str(score2)
    #     print(result2)
        score3=polyclf.score(validX, validY)
        result3="Polynomial accuracy:" +str(score3)
    #     print(result3)
    #     evaluate_print('RBF        |', validY, rbfclf.predict(validX))
    #     evaluate_print('Linear        |', validY, linearclf.predict(validX))
    #     evaluate_print('Polynomial        |', validY, polyclf.predict(validX))
        txtFile = open("../jokeoutput/SVC/RBF_result.txt", 'a')
        txtFile.write(perName+"\n"+result1+'\n\n')
        txtFile.close()

        txtFile = open("../jokeoutput/SVC/Linear_result.txt", 'a')
        txtFile.write(perName+"\n"+result2+'\n\n')
        txtFile.close()

        txtFile = open("../jokeoutput/SVC/Polynomial_result.txt", 'a')
        txtFile.write(perName+"\n"+result3+'\n\n')
        txtFile.close()
    
    return rbfclf,polyclf,linearclf,score1,score2,score3

def Ensemble_models(classifiers,train_data,valid_data,overall):
    X=[]
    Y=[]
    
    for i in train_data:
        arr=train_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            X.append(singleSample)
            Y.append(joke.midjoke)
            
    
    validX=[]
    validY=[]
    perName=''
    for i in valid_data:
        arr=valid_data[i]
        for joke in arr:
            singleSample=[]
            singleSample.append(joke.intensity)
            singleSample.append(joke.pitch)
            singleSample.append(joke.stinten)
            singleSample.append(joke.stpit)
            singleSample.append(joke.max_inten)
            singleSample.append(joke.min_inten)
            singleSample.append(joke.max_pitch)
            singleSample.append(joke.min_pitch)
            validX.append(singleSample)
            validY.append(joke.midjoke)
            perName=joke.performance

    
    clf1 = SimpleClassifierAggregator(classifiers, method='average')
    clf1.fit(X, Y)
#     evaluate_print('Combination by avg   |', validY, clf1.predict(validX))
    
        # combine by median
#     clf2 = SimpleClassifierAggregator(classifiers, method='median')
#     clf2.fit(X, Y)
#     evaluate_print('Combination by median   |', validY, clf2.predict(validX))
    
    clf3 = SimpleClassifierAggregator(classifiers, method='average')
    clf3.fit(X, Y)
    y_predicted = clf3.predict(validX)
    
    diff=0
    for y in zip(validY,y_predicted):
        diff=diff+abs(y[0]-y[1])
    accuracy=1-diff/len(validY)
    
    if(overall==1):
        result="Ensemble accuracy: "+ str(accuracy)
        txtFile = open("../jokeoutput/EnsembleModel/result.txt", 'a')
        txtFile.write(perName+"\n"+result+'\n\n')
        txtFile.close()
    
    return clf1,accuracy

def clear_file():
    txtNames=["../jokeoutput/EnsembleModel/result.txt","../jokeoutput/SVC/Polynomial_result.txt",
             "../jokeoutput/SVC/Linear_result.txt","../jokeoutput/SVC/RBF_result.txt",
             "../jokeoutput/KNN/result.txt","../jokeoutput/decision-tree/forest_result.txt",
             "../jokeoutput/decision-tree/tree_result.txt"]
    for txtName in txtNames:
        if os.path.isfile(txtName):
            os.remove(txtName) 
            
def over_all_model_test(combine_joke_dict):
    perLen=len(perNameDict)
    perNames=list(perNameDict.keys())
    clear_file()
    
    treeTotal,forestTotal,rbfTotal,polyTotal,linearTotal,ensembleTotal,KNNtotal=0,0,0,0,0,0,0
    
    count=0

    for perName in perNames:
        print("validing "+perName)
        
        index=(count+1)%perLen
        train_data2_perName=perNames[index]
        
        train_data,valid_data=leaveOnePerformanceOut(combine_joke_dict,perName)
        train_data1,train_data2=leaveOnePerformanceOut(train_data,train_data2_perName)
        
        tree,forest,treeScore,forestScore=decision_tree(train_data,valid_data,1)

        rbf,poly,linear,rbfScore,polyScore,linearScore=SVC_Algorithm(train_data,valid_data,1)
        
        
        knnModel,KNNscore=KNN(train_data1,train_data2,valid_data,1)

    
        Ensemble_clf,ensemble_score= Ensemble_models(classifier,train_data,valid_data,1)
        
        treeTotal=treeTotal+treeScore
        forestTotal=forestTotal+forestScore
        rbfTotal=rbfTotal+rbfScore
        polyTotal=polyTotal+polyScore
        linearTotal=linearTotal+linearScore
        ensembleTotal=ensembleTotal+ensemble_score
        KNNtotal=KNNtotal+KNNscore
        
        count=count+1
    
    avgTree=treeTotal/perLen
    avgForest=forestTotal/perLen
    avgRBF=rbfTotal/perLen
    avgPoly=polyTotal/perLen
    avglinear=linearTotal/perLen
    avgEnsemble=ensembleTotal/perLen
    avgKNN=KNNtotal/perLen
    
    results=["avgTree "+str(avgTree),"avgForest "+str(avgForest),"avgKNN "+str(avgKNN),"avgRBF "+str(avgRBF),"avgPoly "+str(avgPoly),
            "avglinear "+str(avglinear),"avgEnsemble "+str(avgEnsemble)]
    
    final_ouput=""
    
    for result in results:
        final_ouput=final_ouput+result+'\n'
        print(result)
    
    txtFile = open("../jokeoutput/Final_result_of_allModels.txt", 'w')
    txtFile.write(final_ouput)
    txtFile.close()
    
def jokewise_model_test(jokearr):
    length=len(jokearr)
    if(length<10):
        return
    
    jokedict={}
    jokewise_perNameDict={}
    
    human_rate=[]
    jokename=''
    for joke in jokearr:  
        jokename=joke.jokeName
        pername=joke.performance
        jokewise_perNameDict[pername]=pername
        human_rate.append(joke.midjoke)
#         print(jokename,pername)
    jokedict[jokename]=jokearr
    perLen=len(jokewise_perNameDict)
    perNames=list(jokewise_perNameDict.keys())
    
    print(len(perNames))
    
    txtpath="../jokeoutput/jokewise_all_models/result.txt"
    
    txtFile = open(txtpath, 'a')
    txtFile.write(jokename+'\n')
    txtFile.write("Total performnace for this joke :"+str(len(perNames))+'\n')
    
    treeTotal,forestTotal,rbfTotal,polyTotal,linearTotal,ensembleTotal,KNNtotal=0,0,0,0,0,0,0
    
    count=0

    for perName in jokewise_perNameDict:
        print("validing "+perName)
        
        
        index=(count+1)%perLen
        train_data2_perName=perNames[index]
        
        train_data,valid_data=leaveOnePerformanceOut(jokedict,perName)
        train_data1,train_data2=leaveOnePerformanceOut(train_data,train_data2_perName)
        
        tree,forest,treeScore,forestScore=decision_tree(train_data,valid_data,0)

        rbf,poly,linear,rbfScore,polyScore,linearScore=SVC_Algorithm(train_data,valid_data,0)
        
        
        knnModel,KNNscore=KNN(train_data1,train_data2,valid_data,0)

        if(rbf!=None and poly!=None and linear!=None):
            classifier=[forest,rbf,poly,linear]
        else:
            classifier=[forest,knnModel]
            
        Ensemble_clf,ensemble_score= Ensemble_models(classifier,train_data,valid_data,0)
        
        treeTotal=treeTotal+treeScore
        forestTotal=forestTotal+forestScore
        if(rbfScore!=None):
            rbfTotal=rbfTotal+rbfScore
        if(polyScore!=None):
            polyTotal=polyTotal+polyScore
        if(linearScore!=None):
            linearTotal=linearTotal+linearScore
        ensembleTotal=ensembleTotal+ensemble_score
        KNNtotal=KNNtotal+KNNscore
        
        count=count+1
    
    avgTree=treeTotal/perLen
    avgForest=forestTotal/perLen
    avgRBF=rbfTotal/perLen
    avgPoly=polyTotal/perLen
    avglinear=linearTotal/perLen
    avgEnsemble=ensembleTotal/perLen
    avgKNN=KNNtotal/perLen
    
    results=["avgTree "+str(avgTree),"avgForest "+str(avgForest),"avgKNN "+str(avgKNN),"avgRBF "+str(avgRBF),"avgPoly "+str(avgPoly),
            "avglinear "+str(avglinear),"avgEnsemble "+str(avgEnsemble)]
    
    final_ouput=""
    
    for result in results:
        final_ouput=final_ouput+result+'\n'
        print(result)
    

    txtFile.write(final_ouput+'\n\n')
    txtFile.close()
    
def main():
    global jokeNameDict
    global humanRateDict
    global perNameDict
    perNameDict={}
    humanRateDict=humanAnnotation(r"E:\CapstoneWork\Mid-Joke Laughter Annotations\buitim_Annotations_txt.txt")
    jokeNameDict=getJokeNameForEachPerformance(r"E:\CapstoneWork\k-means-test_by_Tony\joke_name_matching")
    #i am using directory pitch_inten_avg_std_minmax since it is the test contain most information
    combine_joke_dict=findAlltxtAndCombine(r"E:\CapstoneWork\k-means-test_by_Tony\jokeinput")
#     for x in combine_joke_dict:
#         for joke in combine_joke_dict[x]:
#             print(joke.performance,joke.id)
    #print(len(combine_joke_dict))
#     for x in combine_joke_dict:
#         clusterDataIn2D(combine_joke_dict[x],1)
#     for x in combine_joke_dict:
#         clusterDataIn4D(combine_joke_dict[x])
#     for x in combine_joke_dict:
#         clusterDataIn8D(combine_joke_dict[x])
#     overallCorrelation(combine_joke_dict)
#     jokeWiseCorrelation(combine_joke_dict)
#     overallBoxPlot(combine_joke_dict)
#     jokeWiseBoxPlot(combine_joke_dict)

#     over_all_model_test(combine_joke_dict)
    
    txtName="../jokeoutput/jokewise_all_models/result.txt"
    if os.path.isfile(txtName):
            os.remove(txtName) 
    for name in combine_joke_dict:
        jokewise_model_test(combine_joke_dict[name])
    

    

            
main()
print("done")


D:\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


18
validing 2019-04-13 Cienna Nerdy Show at The Drake
validing 2019-09-05 Stand-up Science
validing 2019-09-06 RoboCom
validing 2019-09-19 Bombs Away Cafe
validing 2019-09-21 Laugh Track Town USA
validing 2019-10-11 Singu-hilarity
validing 2019-11-29 Comedy the Musical
validing 2019-11-29 Crapshoot
validing 2019-12-06 Silent Background Recording
validing 2019-12-09 Singu-Hilarity in San Francisco
validing 2019-04-18 Bombs Away Cafe
validing 2019-04-19 Singu-hilarity
validing 2019-04-22 Class Performance
validing 2019-05-16 Bombs Away Cafe
validing 2019-06-19 Trek Theater
validing 2019-06-20 Bombs Away Cafe
validing 2019-08-15 Bombs Away Cafe
validing 2019-08-23 Spectrum
avgTree 0.4444444444444444
avgForest 0.5555555555555556
avgKNN 0.2777777777777778
avgRBF 0.0
avgPoly 0.0
avglinear 0.0
avgEnsemble 0.6111111111111112
15
validing 2019-04-13 Cienna Nerdy Show at The Drake
validing 2019-09-05 Stand-up Science
validing 2019-09-06 RoboCom
validing 2019-09-21 Laugh Track Town USA
validing 20

validing 2019-12-06 Silent Background Recording
validing 2019-04-18 Bombs Away Cafe
validing 2019-04-19 Singu-hilarity
validing 2019-06-19 Trek Theater
validing 2019-08-15 Bombs Away Cafe
validing 2019-08-23 Spectrum
avgTree 0.9230769230769231
avgForest 0.9230769230769231
avgKNN 0.9230769230769231
avgRBF 0.0
avgPoly 0.0
avglinear 0.0
avgEnsemble 0.9230769230769231
11
validing 2019-04-13 Cienna Nerdy Show at The Drake
validing 2019-09-05 Stand-up Science
validing 2019-09-06 RoboCom
validing 2019-12-06 Silent Background Recording
validing 2019-12-09 Singu-Hilarity in San Francisco
validing 2019-04-19 Singu-hilarity
validing 2019-05-16 Bombs Away Cafe
validing 2019-06-19 Trek Theater
validing 2019-06-20 Bombs Away Cafe
validing 2019-08-15 Bombs Away Cafe
validing 2019-08-23 Spectrum
avgTree 0.45454545454545453
avgForest 0.2727272727272727
avgKNN 0.36363636363636365
avgRBF 0.0
avgPoly 0.0
avglinear 0.0
avgEnsemble 0.6363636363636364
15
validing 2019-04-13 Cienna Nerdy Show at The Drake
val

avgTree 0.7
avgForest 0.7
avgKNN 0.6
avgRBF 0.0
avgPoly 0.0
avglinear 0.0
avgEnsemble 0.7
done
